#### Heart Disease Model (WIP)

Descrição: 

In [2]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


### Carregando o Dataset

In [42]:
# --- 1. Carregar e Inspecionar o Dataset ---
print("--- Carregando e Inspecionando o Dataset Dermatology ---")
heart_disease = fetch_ucirepo(id=45)
X_original = heart_disease.data.features
y_original = heart_disease.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
#print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
#print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")
print("-" * 50)

--- Carregando e Inspecionando o Dataset Dermatology ---
Dimensão inicial de X: (303, 13)

Classes de y:
num
0    164
1     55
2     36
3     35
4     13
Name: count, dtype: int64
--------------------------------------------------


### Pré-processamento

In [4]:
print(X_original.isnull().sum()[X_original.isnull().sum() > 0])
X_original = X_original.replace('?', np.nan)
X_original['ca'] = X_original['ca'].fillna(X_original['ca'].mode()[0])
X_original['thal'] = X_original['thal'].fillna(X_original['thal'].mode()[0])

print("\nValores ausentes após preenchimento:")
print(X_original.isnull().sum()[X_original.isnull().sum() > 0])


ca      4
thal    2
dtype: int64

Valores ausentes após preenchimento:
Series([], dtype: int64)


In [75]:
# --- 3. Divisão dos Dados em Treino e Teste ---
from module import termometro
print("--- 3. Binarização dos Atributos para o WiSARD ---")
X_encoded_parts = []
n_bits = 16 # Ex: 8, 16 ou 32 bits

numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Binarizar colunas numéricas contínuas com Termômetro
for col in numerical_cols:
    X_encoded_parts.append(termometro.codificador_termometro(X_original[col].astype(float), n_bits))
    print(f"Coluna '{col}' binarizada com {n_bits} bits (Termômetro).")

# Binarizar colunas categóricas com OneHotEncoder
for col in categorical_cols:
    # OneHotEncoder espera 2D array, então reshape a coluna
    onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_col = onehot_encoder.fit_transform(X_original[[col]].astype(str))
    X_encoded_parts.append(encoded_col.astype(np.uint8))
    print(f"Coluna '{col}' binarizada com OneHotEncoder (Dimensão: {encoded_col.shape[1]} bits).")

X_final_binary = np.hstack(X_encoded_parts)
X_encoded_uint8 = X_final_binary.astype(np.uint8)

--- 3. Binarização dos Atributos para o WiSARD ---
Coluna 'age' binarizada com 16 bits (Termômetro).
Coluna 'trestbps' binarizada com 16 bits (Termômetro).
Coluna 'chol' binarizada com 16 bits (Termômetro).
Coluna 'thalach' binarizada com 16 bits (Termômetro).
Coluna 'oldpeak' binarizada com 16 bits (Termômetro).
Coluna 'sex' binarizada com OneHotEncoder (Dimensão: 2 bits).
Coluna 'cp' binarizada com OneHotEncoder (Dimensão: 4 bits).
Coluna 'fbs' binarizada com OneHotEncoder (Dimensão: 2 bits).
Coluna 'restecg' binarizada com OneHotEncoder (Dimensão: 3 bits).
Coluna 'exang' binarizada com OneHotEncoder (Dimensão: 2 bits).
Coluna 'slope' binarizada com OneHotEncoder (Dimensão: 3 bits).
Coluna 'ca' binarizada com OneHotEncoder (Dimensão: 5 bits).
Coluna 'thal' binarizada com OneHotEncoder (Dimensão: 4 bits).


In [76]:
X_final_binary = np.hstack(X_encoded_parts)
X_encoded_uint8 = X_final_binary.astype(np.uint8)

print(f"\nDimensão de X após todas as codificações: {X_encoded_uint8.shape}")
print(f"Tipo dos elementos de X_final_binary: {X_encoded_uint8.dtype}")
print("Primeiras 2 linhas de X_final_binary:\n", X_encoded_uint8[:2])
print("-" * 50)


Dimensão de X após todas as codificações: (303, 105)
Tipo dos elementos de X_final_binary: uint8
Primeiras 2 linhas de X_final_binary:
 [[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1
  0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0
  0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1
  1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0]]
--------------------------------------------------


In [77]:
# Converter os rótulos de classe para string e depois para uma lista
y_str = y_original.astype(str).tolist()
print(f"\nPrimeiros 5 rótulos de y (como strings): {y_str[:5]}")
print("-" * 50)



Primeiros 5 rótulos de y (como strings): ['0', '2', '1', '0', '0']
--------------------------------------------------


In [78]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final_binary, y_str, test_size=0.3, random_state=42, stratify=y_str
)

In [82]:
# --- 4. Configuração e Treinamento do Modelo WiSARD ---
print("--- 4. Configurando e Treinando o Modelo WiSARD ---")

# addressSize (tuple_size) para o WiSARD.
# O número de bits de entrada de X_encoded_dense para Car Evaluation é 21.
# Um addressSize de 21 ainda é muito grande.
# Escolha um addressSize menor, como 2, 3 ou 4.
# Por exemplo, se X_encoded_dense tem 21 bits, e addressSize=3, teremos 21/3 = 7 RAMs por classe.
addressSize = 8 # Ajuste este valor. 2 ou 3 são bons pontos de partida.

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)
print("-" * 50)


# --- 5. Classificação e Avaliação do Modelo WiSARD ---
print("--- 5. Classificação e Avaliação do Modelo WiSARD ---")

predictions_str = wsd.classify(X_test)

# Convertendo as previsões de string de volta para um array NumPy para fácil comparação
predictions_np = np.array(predictions_str)
y_test_np = np.array(y_test) # Converte o y_test também para numpy array para comparação elemento a elemento

print("Relatório de Classificação:")
print(classification_report(
    y_test_np, predictions_np,
    labels=np.unique(y_test)
))
# print("Matriz de Confusão:")
# print(confusion_matrix(y_test, y_pred))

--- 4. Configurando e Treinando o Modelo WiSARD ---
WiSARD inicializado com addressSize=8
training 55 of 212--------------------------------------------------
--- 5. Classificação e Avaliação do Modelo WiSARD ---
Relatório de Classificação:
training 212 of 212
classifying 91 of 91
              precision    recall  f1-score   support

           0       0.71      0.94      0.81        49
           1       0.18      0.12      0.14        17
           2       0.23      0.27      0.25        11
           3       1.00      0.10      0.18        10
           4       0.00      0.00      0.00         4

    accuracy                           0.57        91
   macro avg       0.42      0.29      0.28        91
weighted avg       0.55      0.57      0.51        91

